# Recipies based bot powered by llamaindex, HuggingFace (for embedding model) and using AWS Bedrock

This notebook shows you how to use `ReAct` Agent over defined tools and a RAG pipeline with Claude 3 Sonnet. Key components used are -

- Reranker model (mixedbread-ai/mxbai-rerank-base-v1)
- Vector Database (AWS OpenSearch)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (mixedbread-ai/mxbai-embed-large-v1)
- Datasets ( Recipies - link , Calorie Count - link)


In [1]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch
%pip install llama-index-embeddings-huggingface


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.7.1
Uninstalling pydantic-2.7.1:
  Successfully uninstalled pydantic-2.7.1
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank

In [3]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ------------------------------------------------------------------------
# LlamaIndex - Amazon Bedrock


llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
#embed_model = BedrockEmbedding(model = "cohere.embed-english-v3")
embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

#BAAI/bge-large-en-v1.5
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()


Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.text_splitter = text_splitter


from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from os import getenv
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)
from llama_index.core import VectorStoreIndex, StorageContext

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "https://vpc-llamaindex-ank67ovnioagxtgqsqlq6iaqwa.us-west-2.es.amazonaws.com")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "gpt-index-huggingface-text-v1")

In [5]:
%pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank
postprocessor = SentenceTransformerRerank(
    model="mixedbread-ai/mxbai-rerank-large-v1", top_n=3
)

Note: you may need to restart the kernel to use updated packages.


In [6]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
calorie_docs = SimpleDirectoryReader(input_dir="./recipies-data/calories").load_data(num_workers=8)

nodes = node_parser.get_nodes_from_documents(documents=calorie_docs,storage_context=storage_context )
base_nodes = text_splitter.get_nodes_from_documents(documents=calorie_docs,storage_context=storage_context )


# initialize an index using our sample data and the client we just created

from llama_index.core import VectorStoreIndex

calories_index = VectorStoreIndex(nodes)

calories_base_index = VectorStoreIndex(base_nodes)

#calories_index = VectorStoreIndex.from_documents(
#    documents=calorie_docs, storage_context=storage_context
#)

#calorie_engine = calories_index.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/llama_index/core/readers/file/base.py:538: UserWarning: Specified num_workers exceed number of CPUs in the system. Setting `num_workers` down to the maximum CPU count.
  warnings.warn(


In [7]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

calorie_engine = calories_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

window_response = calorie_engine.query(
    "How many calories in Egg and Eggnog?"
)
print(window_response)


#res = calorie_engine.query("How many calories in Egg and Eggnog")
#res.response

According to the context information provided:

Egg: 100g of egg contains 97 calories.
Eggnog: 100ml of eggnog contains 88 calories.


In [8]:
window_response = calorie_engine.query(
    "How many calories in Chicken and Chicken Breast?"
)
print(window_response)


According to the context information provided:

Meat, Chicken, 100g, 219 cal, 920 kJ
Meat, Chicken Breast, 100g, 172 cal, 722 kJ

So for 100g:
Chicken contains 219 calories
Chicken Breast contains 172 calories


In [9]:
from llama_index.core.node_parser import TokenTextSplitter
text_splitter = TokenTextSplitter()
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024
Settings.chunk_overlap=20
Settings.text_splitter = text_splitter

In [10]:


from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
recipies_index = VectorStoreIndex.from_documents(
    documents=recipies_docs, storage_context=storage_context
)

recipies_engine = recipies_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

#recipie_engine = recipies_index.as_query_engine(
#    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
 #   node_postprocessors=[
 #       MetadataReplacementPostProcessor(target_metadata_key="window")
 #   ],
#)



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fc324927610>


In [11]:
# run query
#query_engine = index_cohere.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])
res = recipies_engine.query("How can we make pasta?")
res.response

'Here are the steps to make fresh pasta from the context information provided:\n\n1. Knead flour and eggs together to form a firm, non-sticky dough. Knead for 8-10 minutes until smooth and elastic. Cover and let rest for 1 hour.\n\n2. Divide the dough into 8 pieces and flatten each into a rough rectangle. \n\n3. Using a pasta machine, feed each rectangle through the rollers starting at the widest setting. Fold and feed through several times, dusting with flour as needed to prevent sticking.\n\n4. Gradually reduce the roller setting to the second narrowest, feeding the dough through once at each setting without folding to create long smooth sheets.\n\n5. Allow the pasta sheets to dry for about 15 minutes until leathery but still pliable. \n\n6. Attach the fettuccine cutter and feed the dried sheets through to cut into 1/4-inch wide strips.\n\n7. Allow the cut fettuccine to dry for at least 5 more minutes before cooking in salted boiling water for about 2 minutes until tender.\n\nThe con

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=calorie_engine,
        metadata=ToolMetadata(
            name="calorie_metadata",
            description=(
                "Provides information about calories across dairy, fruits and vegetables. You can use this tool to find out calories of different ingridients in a recipe."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=recipies_engine,
        metadata=ToolMetadata(
            name="recipies",
            description=(
                "Provides information about food recipies of various dishes. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### With ReAct Agent

In [13]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill fish dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find recipes for grilled fish dishes with less than 500 calories.
Action: recipies
Action Input: {'input': 'Give me some options for grilled fish dishes that have less than 500 calories'}
Observation: Here are some options for grilled fish dishes that have less than 500 calories:

1. Grilled Salmon with Black Beans and Piment d'Espelette Mayonnaise: This dish features grilled salmon fillets served with a flavorful black bean mixture and a spicy mayonnaise. The black beans and vegetables help keep the calorie count reasonable.

2. Seared Tuna with Olive-Tapenade Vinaigrette and Arugula: Seared tuna steaks are served over a bed of arugula with a zesty olive-tapenade vinaigrette. The vinaigrette adds flavor without excessive calories.

3. Grilled Halibut with Lima Bean and Roasted Tomato Sauce: Grilled halibut is a lean, low-calorie fish option, and it's served with a fresh lima bean and roasted tomato 

In [14]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("How many calories in sliced gouda cheese?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find the calorie information for sliced gouda cheese.
Action: calorie_metadata
Action Input: {"input": "How many calories are in sliced gouda cheese?"}

Observation: Here are the calorie details for sliced gouda cheese:

Gouda cheese has around 100-120 calories per 1 oz (28g) slice. The exact number of calories can vary slightly depending on the fat content, which ranges from around 25-30% for gouda cheese.

Some key calorie information for gouda cheese:

- 1 oz (28g) slice of regular gouda cheese: Around 100-110 calories
- 1 oz (28g) slice of low-fat gouda cheese: Around 70-80 calories
- 1 cup (112g) shredded regular gouda cheese: Around 450 calories
- 1 cup (112g) shredded low-fat gouda cheese: Around 300 calories

The calories mainly come from the fat and protein content in the cheese. Gouda is a semi-soft cheese, so it has a moderate amount of fat compared to harder aged cheeses.

Thought: I now 

In [26]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill chicken dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find recipes and calorie information for grilled chicken dishes under 500 calories.
Action: recipies
Action Input: {'input': 'grilled chicken recipes under 500 calories'}
Observation: Here is a grilled chicken recipe under 500 calories from the provided context:

Maple-Soy Barbecue Grilled Chicken

Ingredients:
- Olive oil
- Garlic cloves
- Tomato paste
- Maple syrup
- Soy sauce
- Worcestershire sauce  
- Rice vinegar
- Sriracha or hot sauce
- Whole chicken or bone-in chicken pieces  
- Kosher salt

Instructions:
1. Make the barbecue sauce by cooking garlic, tomato paste, maple syrup, soy sauce, Worcestershire, vinegar, and Sriracha.
2. Season the chicken generously with salt. 
3. Grill the chicken over direct heat to brown on all sides.
4. Move chicken to indirect heat, cover and grill until cooked through, basting with the barbecue sauce.

This grilled chicken recipe uses a flavorful homemade maple

In [16]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a morning breakfast smoothie which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find calorie information for different ingredients to create a low-calorie smoothie.
Action: calorie_metadata
Action Input: {'input': 'What are the calorie counts for common smoothie ingredients like fruits, vegetables, dairy products, etc?'}
Observation: Based on the context information provided, here are some relevant calorie counts for common smoothie ingredients:

Fruits:
Banana - 100g = 89 cal
Strawberries - 100g = 32 cal  
Blueberries - 100g = 57 cal
Mango - 100g = 60 cal
Pineapple - 100g = 50 cal

Vegetables: 
Spinach - 100g = 23 cal
Kale - 100g = 49 cal

Dairy Products:
Milk - 100ml = 61 cal
Plain Yogurt - 100g = 61 cal
Soy Milk - 100ml = 45 cal

Other Ingredients:
Peanut Butter - 100g = 589 cal
Almond Butter - 100g = 614 cal
Honey - 100g = 304 cal
Oats - 100g = 384 cal

These calorie counts can help estimate the calories in a smoothie based on the ingredients and portions used. The fruits an

In [17]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("I want to make a cheesecake which has fruits as the topping and is not more than 2000 calories? Think Step by Step and recommend.")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find a cheesecake recipe with fruits as topping and calorie information.
Action: recipies
Action Input: {'input': 'cheesecake recipe with fruits as topping'}
Observation: Here is a delicious cheesecake recipe with a caramel apple topping:

Creole Cream Cheesecake with Caramel Apple Topping

Ingredients:

For the Crust:
- 2 cups graham cracker crumbs
- 3 tablespoons granulated sugar
- 3 tablespoons brown sugar
- 1/4 teaspoon kosher salt
- 1/2 cup unsalted butter, melted

For the Filling: 
- 2 pounds cream cheese, at room temperature
- 1 cup granulated sugar
- 1 1/3 cups Creole Cream Cheese (or 1 cup sour cream mixed with 1/4 cup buttermilk)
- 1 teaspoon vanilla bean paste
- 2 eggs, at room temperature

For the Caramel Apple Topping:
- 3 3/4 tablespoons unsalted butter
- 3/4 cup granulated sugar
- 1/4 cup brown sugar
- 1 1/2 tablespoons honey
- 4 cups peeled, cored, chopped Granny Smith apples
- 1 1/2 teaspoon

In [25]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend me a cocktail drink which is based on Egg Nog? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find a cocktail recipe based on egg nog and get the calorie information.
Action: recipies
Action Input: {"input": "Recommend a cocktail drink recipe based on egg nog"}

Observation: Here is a popular cocktail recipe using egg nog:

Egg Nog Martini

Ingredients:
- 2 oz vodka
- 4 oz egg nog
- 1 oz coffee liqueur (such as Kahlua)
- Nutmeg for garnish

Instructions:
1. Fill a cocktail shaker with ice cubes. 
2. Add the vodka, egg nog and coffee liqueur.
3. Shake vigorously for 10-15 seconds until well-chilled.
4. Strain into a chilled martini glass.
5. Garnish with a sprinkle of freshly grated nutmeg.

This creamy and indulgent cocktail combines the richness of egg nog with vodka and coffee liqueur for an adults-only holiday treat.

Thought: The recipe tool provided a cocktail recipe using egg nog, but I still need calorie information to fully answer the question.
Action: calorie_metadata
Action Input: {"input":

In [19]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog based cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find eggnog cocktail recipes and their calorie counts.

Action: recipies
Action Input: {"input": "eggnog cocktail recipes"}

Observation: Here are some popular eggnog cocktail recipes:

1. Eggnog White Russian: Eggnog, vodka, coffee liqueur
2. Spiked Eggnog: Eggnog, rum, brandy, nutmeg 
3. Eggnog Martini: Eggnog, vodka, coffee liqueur, cinnamon
4. Bourbon Eggnog: Eggnog, bourbon, nutmeg
5. Peppermint Eggnog: Eggnog, peppermint schnapps, crushed candy canes

Action: calorie_metadata
Action Input: {"input": "calorie counts for eggnog, vodka, coffee liqueur, rum, brandy, bourbon, peppermint schnapps"}

Observation: Here are the approximate calorie counts for the ingredients:

- Eggnog (1 cup): 340 calories
- Vodka (1.5 oz): 97 calories  
- Coffee liqueur (1 oz): 115 calories
- Rum (1.5 oz): 115 calories
- Brandy (1.5 oz): 120 calories
- Bourbon (1.5 oz): 105 calories
- Peppermint schnapps (1 oz): 100 calories



In [20]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find recipes for eggnog cocktails and their calorie counts.
Action: recipies
Action Input: {'input': 'eggnog cocktail recipes'}
Observation: Here are a couple eggnog cocktail recipes based on the context information provided:

Spiced Apple Cake with Eggnog Sauce:
This recipe calls for serving a spiced apple cake accompanied by chilled eggnog as a sauce or accompaniment. The cake contains ingredients like raisins, apples, pecans, crystallized ginger, and warm spices like allspice, cinnamon, nutmeg and cloves.

Cosmopolitan Cranberry Sauce:  
While not a traditional eggnog cocktail, this festive cranberry sauce contains vodka and orange liqueur like Grand Marnier or Triple Sec. It could be an interesting twist to serve it alongside or mixed into eggnog for an adults-only cocktail.
Thought: The recipes provided give some ideas for eggnog cocktails, but do not include specific calorie counts. To fully answer the

In [21]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find a good Indian curry dish recipe and its calorie information.
Action: recipies
Action Input: {'input': 'Recommend a good Indian curry dish recipe'}
Observation: Here is a delicious Indian curry dish recipe to try:

Prawn Moilee (South Indian Shrimp Curry with Coconut)

Ingredients:
- 6 green chilies
- Vegetable oil
- Mustard seeds  
- Fresh curry leaves
- Onions, sliced
- Garlic paste
- Ginger paste
- Salt
- Black pepper
- Turmeric
- Fresh ginger, cut into matchsticks  
- Coconut milk
- Coconut cream
- Large prawns/shrimp
- Tomatoes, cut into wedges
- Lemon wedges for serving

Instructions:
1. Sauté garlic, ginger, salt, pepper, turmeric, chilies and ginger matchsticks in oil. 
2. Pour in coconut milk and cream and simmer for 20 minutes.
3. Fry some extra curry leaves until crisp to use as a garnish.
4. Add prawns and tomatoes to the sauce and simmer until prawns are cooked through.
5. Serve the 

In [22]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish which is low on calories and good on fiber? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find a low-calorie, high-fiber Indian curry dish and get calorie counts for the ingredients.
Action: recipies
Action Input: {'input': 'Recommend a low-calorie, high-fiber Indian curry dish'}
Observation: A great low-calorie, high-fiber Indian curry dish is a lentil or dal curry. Lentils are packed with fiber, protein, and nutrients while being low in calories. You can make a flavorful dal by simmering lentils with aromatic spices like cumin, coriander, turmeric, and ginger. Stir in some tomatoes, spinach or other greens for extra nutrition. Serve the dal over brown rice or with whole wheat naan for an incredibly satisfying yet light and healthy meal.
Thought: The recipe recommendation for a lentil or dal curry sounds like a good option for a low-calorie, high-fiber Indian dish. To get the calorie counts for the ingredients, I will use the calorie_metadata tool.
Action: calorie_metadata
Action Input: 